# Data Processing

## Import modules

In [ ]:
# import SparkSession
from pyspark.sql import SparkSession

In [ ]:
import pyspark.sql.functions as fn
from pyspark.sql.types import StringType,DoubleType,IntegerType

## Set spark session

In [ ]:
spark.sparkContext.appName

## Load data

In [ ]:
# Load csv Dataset 
df=spark.read.csv('data/sample_data.csv',inferSchema=True,header=True)
df.createOrReplaceTempView("dfTable")

## Inspect data

In [ ]:
# columns of dataframe
df.columns

In [ ]:
# shape of dataset
df.count(),len(df.columns)

In [ ]:
# print dataframe schema
df.printSchema()

In [ ]:
# display fisrt few rows of dataframe
df.show()
#df.show(10)

## Grouping data

In [ ]:
# group by one column
df.groupBy('mobile').count().show(5)

In [ ]:
# sort value counts
df.groupBy('mobile').count().orderBy('count',ascending=False).show(5)

In [ ]:
# calculate statistical measures
df.groupBy('mobile').mean().show(5)

In [ ]:
# calculate statistical measures
df.groupBy('mobile').sum().show()

In [ ]:
# calculate statistical measures
df.groupBy('mobile').max().show()

In [ ]:
# calculate statistical measures
df.groupBy('mobile').min().show()

In [ ]:
# use spark sql
spark.sql('''select mobile, count(*) as count from dfTable
        group by mobile''').show()

In [ ]:
# use spark sql
spark.sql('''select mobile, min(experience), min(age) from dfTable
        group by mobile''').show()

In [ ]:
# Aggregation
df.groupBy('mobile').agg({'experience':'sum'}).show()

## Load and Inspect data

In [ ]:
# Load csv Dataset
rtdf=spark.read.csv('data/online_retail_dataset.csv',inferSchema=True,header=True)
rtdf.createOrReplaceTempView("rtTable")

In [ ]:
# columns of dataframe
rtdf.columns

In [ ]:
# shape of dataset
rtdf.count(),len(rtdf.columns)

In [ ]:
# print dataframe schema
rtdf.printSchema()

In [ ]:
# display fisrt few rows of dataframe
rtdf.show()
#rtdf.show(10)

In [ ]:
# counting
rtdf.select(fn.count('StockCode')).show() 

In [ ]:
# distinct count
rtdf.select(fn.countDistinct('StockCode')).show()

In [ ]:
# get minimun and maximun
rtdf.select(fn.min("Quantity"), fn.max("Quantity")).show()

In [ ]:
# Variance and Standard Deviation
rtdf.select(fn.var_pop('Quantity'), fn.var_samp('Quantity'),
        fn.stddev_pop('Quantity'), fn.stddev_samp('Quantity')).show()

In [ ]:
# use sql
spark.sql('''select count(StockCode) from rtTable''').show()

In [ ]:
# Covariance and Correlation
rtdf.select(fn.corr('InvoiceNo', 'Quantity'), fn.covar_samp('InvoiceNo', 'Quantity'),
        fn.covar_pop('InvoiceNo', 'Quantity')).show()

In [ ]:
# count with groupby
rtdf.groupBy("InvoiceNo", "CustomerId").count().show(5)

In [ ]:
# agg function
rtdf.groupBy('InvoiceNo').agg({'Quantity':'count'}).show(5)

In [ ]:
# agg function
rtdf.groupBy('InvoiceNo').agg(fn.count('Quantity').alias('quan'),
        fn.expr('count(Quantity)')).show(5)

In [ ]:
# agg function
rtdf.groupBy('InvoiceNo').agg({'Quantity':'min', 'UnitPrice':'max'}).show(5)

In [ ]:
# agg function
rtdf.groupBy('InvoiceNo').agg(fn.max('Quantity'),
        fn.min('Quantity')).show(5)

## UDF

In [ ]:
# UDF
from pyspark.sql.functions import udf

In [ ]:
# normal function 
def price_range(brand):
    if brand in ['Samsung','Apple']:
        return 'High Price'
    elif brand =='MI':
        return 'Mid Price'
    else:
        return 'Low Price'

In [ ]:
# create udf using python function
brand_udf=udf(price_range,StringType())

In [ ]:
# apply udf on dataframe
df.withColumn('price_range',brand_udf(df['mobile'])).show(10,False)

In [ ]:
# using lambda function
age_udf = udf(lambda age: "young" if age <= 30 else "senior", StringType())

In [ ]:
# apply udf on dataframe
df.withColumn("age_group", age_udf(df.age)).show(10)

## Saving file

In [ ]:
# save the dataframe as single csv 
df.coalesce(1).write.csv('data/df_data.csv', header='True', mode='overwrite')

In [ ]:
# save the data into parquet format 
rtdf.write.parquet('data/retail_dataset_parquet', mode='overwrite')

In [ ]:
# read the data from parquet format 
rtdf2=spark.read.parquet('data/retail_dataset_parquet')

In [ ]:
rtdf2.show(10)

## Outlier

In [ ]:
import numpy as np

In [ ]:
wdf = spark.read.csv('data/winequality_white.csv',sep=';',inferSchema=True,header=True)

In [ ]:
# columns of dataframe
wdf.columns

In [ ]:
# shape of dataset
wdf.count(),len(wdf.columns)

In [ ]:
# print dataframe schema
wdf.printSchema()

In [ ]:
# display fisrt few rows of dataframe
#wdf.show()
wdf.show(10)

In [ ]:
wdf.select('pH','sulphates','chlorides').summary().show()

In [ ]:
bounds = {
    c: dict(
        zip(["q1", "q3"], wdf.approxQuantile(c, [0.25, 0.75], 0))
    )
    for c in ['pH']
}

In [ ]:
bounds

In [ ]:
for c in bounds:
    iqr = bounds[c]['q3'] - bounds[c]['q1']
    bounds[c]['lower'] = bounds[c]['q1'] - (iqr * 1.5)
    bounds[c]['upper'] = bounds[c]['q3'] + (iqr * 1.5)

In [ ]:
bounds

In [ ]:
wdf2 = wdf.select(
    '*',
    *[
        fn.when(
            fn.col(c).between(bounds[c]['lower'], bounds[c]['upper']),
            0
        ).otherwise(1).alias(c+"_out") 
        for c in ['pH']
    ]
)

In [ ]:
wdf2.select('pH', 'pH_out').show(10)

In [ ]:
wdf2.select('pH','pH_out').filter('pH_out==1').show(10)

In [ ]:
wdf2.select('pH','pH_out').where('pH_out == 1').count()

In [ ]:
wdf2.createOrReplaceTempView("wdf2Table")

In [ ]:
spark.sql('''select pH,pH_out from wdf2Table where pH_out=1 limit 10''').show()

In [ ]:
# save the data into parquet format 
wdf2.write.csv('data/wdf2', header='True', mode='overwrite')

In [ ]:
# Load csv Dataset
wdf3=spark.read.csv('data/wdf2',inferSchema=True,header=True)
wdf3.createOrReplaceTempView("wdf4Table")

In [ ]:
wdf3.filter('pH_out==1').show(15)